# Thermo-Chemistry Benchmark
This notebook will try to compare the BigDFT approach to other Quantum Chemistry formalisms use the systems of the W4-11 thermochemistry benchmark.

> Karton, Amir, Shauli Daon, and Jan ML Martin. "W4-11: A high-confidence benchmark dataset for computational thermochemistry derived from first-principles W4 data." Chemical Physics Letters 510, no. 4-6 (2011): 165-178.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dataset = "W4-11"

We will explore a few parameters: pseudopotentials (Krack vs. NLCC), functional (PBE vs. PBE0 vs. B3LYP), and basis set (PCSEG series, reference: def2-QZVP). 

## Data Retrieval
Before starting, we have to get the dataset.

In [ ]:
from os import system
from os.path import exists
if not exists("W4-11.tar"):
    system("wget http://www.thch.uni-bonn.de/tc.old/downloads/GMTKN/GMTKN55/W4-11.tar")
    system("wget http://www.thch.uni-bonn.de/tc.old/downloads/GMTKN/GMTKN55/W4-11ref.html")
    system("wget http://www.thch.uni-bonn.de/tc.old/downloads/GMTKN/GMTKN55/CHARGE_MULTIPLICITY_W4-11.txt")
    system("tar -xvf W4-11.tar")
    system("mv W4-11ref.html W4-11/")
    system("mv CHARGE_MULTIPLICITY_W4-11.txt W4-11/")

## Read In The Systems
First we will read in the full set of systems.

In [ ]:
from os.path import join, basename
from glob import glob

names = [basename(x) for x in glob(join(dataset, "*"))]

In [ ]:
from BigDFT.IO import read_xyz

systems = {}
for n in names:
    if n in ["c2", "cloo"]:  # Systems with PSI4 convergence problems
        continue
    try:
        with open(join(dataset, n, "struc.xyz")) as ifile:
            sys = read_xyz(ifile)
    except NotADirectoryError:
        continue
    # Remove Sulfur which doesn't have an Saha
    skip = False
    for frag in sys.values():
        for at in frag:
            if at.sym == "S":
                print("Skip:", n)
                skip = True
                continue
    if not skip:
        systems[n] = sys

We also need to read in the charge and multiplicity data.

In [ ]:
charges = {}
multiplicities = {}
with open(join(dataset, "CHARGE_MULTIPLICITY_" + dataset + ".txt")) as ifile:
    for line in ifile:
        n, c, m = line.split()
        charges[n] = int(c)
        multiplicities[n] = int(m)

Last, we need to read the stochiometry data which is helpfully provided by the GMTKN55 dataset.

In [ ]:
def process_stochiometry():
    from bs4 import BeautifulSoup
    with open(join(dataset, dataset+"ref.html")) as ifile:
        html = ifile.read()
    soup = BeautifulSoup(html, 'html.parser')
    stochiometries = {}
    for row in soup.table:
        lrow = list(row)
        if lrow[0] == '\n':
            continue
        key = lrow[1].get_text().strip()
        stochiometries[key] = {"pairs": [key], "weights": [], "ref": None}
        for i in range(2, len(lrow)):
            val = lrow[i].get_text().strip()
            try:
                stochiometries[key]["weights"].append(int(val))
            except ValueError:
                try:
                    stochiometries[key]["ref"] = float(val)
                except ValueError:
                    val = lrow[i].get_text().strip()
                    if val not in [",", ""]:
                        stochiometries[key]["pairs"].append(val)
    return stochiometries

In [ ]:
stoch = process_stochiometry()

## Calculation Routines
Now we will build the routines that can process the dataset with a given set of parameters.

In [ ]:
def compute_bigdft(systems, charges, mults, pp, functional):
    from BigDFT.Calculators import SystemCalculator
    from BigDFT.Inputfiles import Inputfile
    from copy import deepcopy
    
    cinp = Inputfile()
    cinp.set_xc(functional)
    
    if pp == "Saha":
        cinp.set_psp_nlcc()
        cinp.set_hgrid(0.45)
    elif pp == "Krack":
        cinp.set_psp_krack()
        cinp.set_hgrid(0.37)
    else:
        raise ValueError("Invalid PP")

    calc = SystemCalculator(skip=True, verbose=False)
    
    param_id = str(pp) + "_" + functional
    
    logfiles = {}
    for k, v in systems.items():
        cinp2 = deepcopy(cinp)
        cinp2.charge(charges[k])
        if mults[k] > 1:
            cinp2.spin_polarize(mpol = mults[k]-1)
        logfiles[k] = calc.run(sys=v, input=cinp2, name=k,
                               run_dir="work_bigdft_" + param_id)

    return {k: v.energy for k, v in logfiles.items()}

In [ ]:
def compute_psi4(systems, charges, mults, basis, functional):
    from BigDFT.Interop.PSI4Interop import PSI4Calculator
    from psi4 import set_options, set_num_threads, set_memory
    from os import environ
    
    calc = PSI4Calculator(skip=True, verbose=False)
    options = {"reference": "uhf", 
               "dft_spherical_points": 590,
               "dft_radial_points": 99,
               "scf_type": "direct",
               "maxiter": 1000,
               "df_scf_guess": False}
    set_options(options)
    set_memory('2048 MB')
    set_num_threads(1)
#     set_num_threads(int(environ['OMP_NUM_THREADS']))
    
    param_id = basis + "_" + functional
    
    logfiles = {}
    for k, v in systems.items():
        logfiles[k] = calc.run(sys=v, name=k, action="energy",
                               basis=basis, method=functional,
                               chargeA=charges[k], multiplicityA=mults[k],
                               run_dir="work_psi4_" + param_id)
    return {k: v["energy"]["total"] for k, v in logfiles.items()}

At this point, we can test out our functions on a small subset of the data.

In [ ]:
subset = {}
for i, (k, v) in enumerate(systems.items()):
    if i > 2:
        break
    subset[k] = v

In [ ]:
energies_bdft = compute_bigdft(systems=subset,
                               charges=charges,
                               mults=multiplicities, 
                               pp="Saha", functional="PBE")

In [ ]:
print(energies_bdft)

In [ ]:
energies_psi4 = compute_psi4(systems=subset, 
                             charges={k: charges[k] for k in subset},
                             mults={k: multiplicities[k] for k in subset}, 
                             basis="pcseg-2", functional="PBE")

In [ ]:
print(energies_psi4)

## Remote Calculations
Now we will perform the full set of calculations on a remote machine for different parameters.

In [ ]:
from remotecomputer.spring import Spring
url = Spring(user="dawson")
url.queue = "winter2"
url.mpi = 1
url.omp = 36
url.conda = "thermal_iop"
remote_dir = "/home/dawson/Runs/2023/thermal"

In [ ]:
from remotemanager.serialisation import serialjsonpickle
serial = serialjsonpickle()

In [ ]:
from remotemanager.dataset import Dataset
bdft_set = Dataset(url=url, function=compute_bigdft, 
                   dbfile="bigdft-db",
                   serialiser=serial,
                   remote_dir=remote_dir)

In [ ]:
psi4_set = Dataset(url=url, function=compute_psi4, 
                   dbfile="psi4-db",
                   serialiser=serial,
                   remote_dir=remote_dir)

BigDFT runs

In [ ]:
for functional in ["PBE", "PBE0", "B3LYP"]:
    for pp in ["Krack", "Saha"]:
        args = {"systems": systems,
                "charges": charges, "mults": multiplicities, 
                "pp": pp, "functional": functional}
        bdft_set.append_run(arguments=args)

In [ ]:
_ = bdft_set.run()

PSI4

In [ ]:
for functional in ["pbe", "pbe0", "b3lyp"]:
    for basis in ["pcseg-0", "pcseg-1", "pcseg-2", "pcseg-3", "def2-QZVP"]:
        args = {"systems": systems, 
                "charges": charges, "mults": multiplicities, 
                "basis": basis, "functional": functional}
        psi4_set.append_run(id={"basis": basis, "functional": functional}, arguments=args)

In [ ]:
_ = psi4_set.run()

Synchronize.

In [ ]:
from time import sleep
while not bdft_set.all_finished: sleep(10)
_ = bdft_set.fetch_results()
while not psi4_set.all_finished: sleep(10)
_ = psi4_set.fetch_results()

## Analyze Results
The last step is to create a plot summarizing the results. First we compute the energies according to the stochiometry rules.

In [ ]:
def compute_atomization(data, stoch):
    atomization = {}
    for point, spec in stoch.items():
        if point not in data:
            continue
        atomization[point] = 0
        for p, w in zip(spec["pairs"], spec["weights"]):
            atomization[point] += w * data[p]
    return atomization

In [ ]:
atomization = {"BigDFT": {}, "PSI4": {}}

i = 0
for functional in ["PBE", "PBE0", "B3LYP"]:
    atomization["BigDFT"][functional] = {}
    for pp in ["Krack", "Saha"]:
        res = bdft_set.results[i]
        atomization["BigDFT"][functional][pp] = compute_atomization(res, stoch)
        i += 1

i = 0
for functional in ["pbe", "pbe0", "b3lyp"]:
    atomization["PSI4"][functional] = {}
    for basis in ["pcseg-0", "pcseg-1", "pcseg-2", "pcseg-3", "def2-QZVP"]:
        res = psi4_set.results[i]
        atomization["PSI4"][functional][basis] = compute_atomization(res, stoch)
        i += 1

# for params, results in zip(bdft_set.ids, bdft_set.fetch_results()):
#     atomization["BigDFT"][(params["nlcc"], params["functional"])] = \
#       compute_atomization(results, stoch)
    
# for params, results in zip(psi4_set.ids, psi4_set.fetch_results()):
#     atomization["PSI4"][(params["basis"], params["functional"])] = \
#       compute_atomization(results, stoch)

Now plot the errors.

In [ ]:
def compute_errors(xvals, yvals):
    return {k: 630*abs(xvals[k] - yvals[k]) for k in order}

Precision.

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(1, 3, figsize=(6, 4))

order = [x for x in stoch if x in systems]

for i, fun in enumerate(["PBE", "PBE0", "B3LYP"]):
    axs[i].set_title(fun, fontsize=12)
    ref = {x: atomization["PSI4"][fun.lower()]["def2-QZVP"][x] for x in order}

    computed = {x: atomization["BigDFT"][fun]["Krack"][x] for x in order}
    errors = [list(compute_errors(computed, ref).values())]
    labels = ["BigDFT - Krack"]

    computed = {x: atomization["BigDFT"][fun]["Saha"][x] for x in order}
    errors += [list(compute_errors(computed, ref).values())]
    labels += ["BigDFT - NLCC"]

    for basis in ["pcseg-0", "pcseg-1", "pcseg-2", "pcseg-3"]:
        computed = {x: atomization["PSI4"][fun.lower()][basis][x] for x in order}
        errors += [list(compute_errors(computed, ref).values())]
        labels += ["PSI4 - " + basis.upper()]

    axs[i].boxplot(errors)
    axs[i].set_xticks(range(1, 1 + len(labels)))
    axs[i].set_xticklabels(labels, rotation=90, fontsize=12)
    axs[i].set_ylim(-10, 180)

axs[0].set_ylabel("Error (kcal/mol)", fontsize=14)
fig.suptitle("Precision (vs. def2-QZVP)", fontsize=14)
fig.tight_layout()
fig.savefig("thermal.png", dpi=300)

Accuracy.

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(1, 3, figsize=(6, 4))

order = [x for x in stoch if x in systems]

for i, fun in enumerate(["PBE", "PBE0", "B3LYP"]):
    axs[i].set_title(fun, fontsize=12)
    ref = {x: stoch[x]["ref"]/630 for x in order}

    computed = {x: atomization["BigDFT"][fun]["Krack"][x] for x in order}
    errors = [list(compute_errors(computed, ref).values())]
    labels = ["BigDFT - Krack"]
    
    edict = compute_errors(computed, ref)
    mkey = max(edict, key=edict.get)
    print("Krack:", mkey, edict[mkey])

    computed = {x: atomization["BigDFT"][fun]["Saha"][x] for x in order}
    errors += [list(compute_errors(computed, ref).values())]
    labels += ["BigDFT - NLCC"]
    
    edict = compute_errors(computed, ref)
    mkey = max(edict, key=edict.get)
    print("NLCC", mkey, edict[mkey])

    for basis in ["pcseg-0", "pcseg-1", "pcseg-2", "pcseg-3"]:
        computed = {x: atomization["PSI4"][fun.lower()][basis][x] for x in order}
        errors += [list(compute_errors(computed, ref).values())]
        labels += ["PSI4 - " + basis.upper()]
        edict = compute_errors(computed, ref)
        mkey = max(edict, key=edict.get)
        print(basis, mkey, edict[mkey])

    axs[i].boxplot(errors)
    axs[i].set_xticks(range(1, 1 + len(labels)))
    axs[i].set_xticklabels(labels, rotation=90, fontsize=12)
    axs[i].set_ylim(-10, 180)

axs[0].set_ylabel("Error (kcal/mol)", fontsize=14)
fig.suptitle("Accuracy (vs. W4 Protocol)", fontsize=14)
fig.tight_layout()
# fig.savefig("thermal.png", dpi=300)